<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np
import pandas as pd
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy import Spider

In [2]:
class craiglistSpider(scrapy.Spider):
    name = "craiglist_surfboard_crawl"
    
    start_urls = [
        'https://honolulu.craigslist.org/search/sss?query=surfboard&sort=rel'
        ]

    allowed_domains = ["craigslist.org"]

    def parse(self, page):
        products = page.xpath('//p[@class="result-info"]')

        for product in products:
            relative_url = product.xpath('a/@href').extract_first()
            absolute_url = page.urljoin(relative_url)
            title = product.xpath('a/text()').extract_first()
            
            yield Request(absolute_url, callback=self.parse_page, meta={'URL': absolute_url, 'Title': title})

        relative_next_url = response.xpath('//a[@class="button next"]/@href').extract_first()
        absolute_next_url = "https://newyork.craigslist.org" + relative_next_url
        
    def parse_page(self, page):
        yield {
        'title': page.xpath('//*[@id="titletextonly"]/text()').extract(),
        'price': page.xpath('/html/body/section/section/h2/span/span[2]/text()').extract(),
        'description': "".join(line for line in response.xpath('//*[@id="postingbody"]/text()').extract()),
        'condition': page.xpath('/html/body/section/section/section/div[1]/p/span[1]/b/text()').extract(),
        'manufacturer': page.xpath('/html/body/section/section/section/div[1]/p/span[2]/b/text()').extract(),
        'model_name': page.xpath('/html/body/section/section/section/div[1]/p/span[3]/b/text()').extract(),
        'size_dimensions': page.xpath('/html/body/section/section/section/div[1]/p/span[4])/b/text()').extract()

    }
                    
        # Getting the information needed to continue to the next ten entries.
#         next_page = response.xpath('//*[@id="searchform"]/div[5]/div[3]/span[2]/span[4]').extract_first()
        
#         # Recursively calling the spider to process the next ten entries, if they exist.
#         if next_page is not None:
#             next_page = '{}&lhcontinue={}'.format(self.start_urls[0],next_page)
#             yield scrapy.Request(next_page, callback=self.parse)
            
    
process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'surfboard.json',
    'ROBOTSTXT_OBEY': False,
    'USER_AGENT': 'ThinkfulDataScienceBootcampCrawler (thinkful.com)',
    'AUTOTHROTTLE_ENABLED': True,
    'HTTPCACHE_ENABLED': True,
    'LOG_ENABLED': False,
    'CLOSESPIDER_PAGECOUNT' : 10
})
                                         

process.crawl(craiglistSpider)
process.start()